In [1]:
data = [(2.0,3.0,1.0),(4.0,4.0,1.0),(1.0,0.5,-1.0),(0.5,1.0,-1.0),(0.2,0.2,-1.0),(4.0,5.0,1.0)]
def load_data(data,index):
    """
        返回data数组中，根据index进行拆分，返回训练集和测试集
    """
    train_data = data[0:index]
    test_data = data[index:]
    return train_data,test_data    

In [2]:
train_data,test_data = load_data(data,4)

In [3]:
print(train_data)
print(test_data)

[(2.0, 3.0, 1.0), (4.0, 4.0, 1.0), (1.0, 0.5, -1.0), (0.5, 1.0, -1.0)]
[(0.2, 0.2, -1.0), (4.0, 5.0, 1.0)]


In [4]:
def load_x_and_label(data):
    """将数据集合中的测试集合数据集进行拆分"""
    x=[]
    label=[]
    for k in data:
        x1= (k[0],k[1])
        x.append(x1)
        label.append(k[2])
    return x,label
x,label = load_x_and_label(train_data)
print (x)
print(label)

[(2.0, 3.0), (4.0, 4.0), (1.0, 0.5), (0.5, 1.0)]
[1.0, 1.0, -1.0, -1.0]


In [5]:
def sums(a):
    """对a求和"""
    s = 0.0
    for i in a:
        s = s + i
    return s
def mul(a,label):
    """a 与label中每一项相乘，返回一个长度相同的数组，
       a = [1,2,3]
       label = [2,2,2]
       mul(a,label) = [ 2,4,6 ]
    """
    result = []
    for i in range(len(a)):
        result.append(a[i]*label[i])
    return result
def calculate(a,x,label):
    """计算给定alpha后得到的函数值，用于选取最大值"""
    s = 0.0
    for i in range(len(x)):
        for j in range(len(x)):
            s = s + a[i]*a[j]*label[i]*label[j]*sums(mul(x[i],x[j]))
    return (sums(a)-1.0/2*s)


def cal_aj(a,i,label,j):
    """计算 aj """
    a[j]=0-a[i]*label[i]*label[j]
    return a[j]

In [6]:
# 给定alpha
a=[0.0,0.0,0.0,0.0]
# 用来标识已经被访问过
used=[0,0,0,0]
# 跨度
length = 10000
# 步长 千分之一
step = 1000.0
# 获取训练集和测试集
x,label = load_x_and_label(train_data)

def train_a(x, label, a, used, length, step):
    """
    x 标识测试数据
    label 标签
    a 标识 alpha
    used 表示 被计算过的数组
    length 表示跨度，在这个0-length的跨度内找最大值
    step 表示步长
    """
    for i in range(0,len(a)):
        max_value = -1
        max_j = 0
        max_index = -1
        if used[i] == 0:
            used[i] = 1
            # 选aj
            for j in range(0,len(a)):
                if j != i and used[j] == 0:
                    used[j] = 1
                    # 确定ai的值
                    for d in range(1,length):
                        a[i]  = d/step
                        oldja = a[j]
                        aj = cal_aj(a,i,label,j)
                        if aj >= 0:
                            k = calculate(a,x,label)
                            if k> max_value:
                                # 选取最大值
                                max_value = k
                                max_index = d
                                max_j = j
                        else:
                            a[j]=oldja
                    if max_index > -1 :
                        # 找到第一个符合条件的aj就可以
                        a[i] = max_index/step
                        a[max_j] = cal_aj(a,i,label,max_j)
                        break
                    else:
                        # 如果没找到 需要回溯
                        used[j]=0
                        #a[j] = oldaj
    return a
train_a(x,label,a,used,length,step)
print(sums(mul(a,label)))
print (a)

8.673617379884035e-19
[0.276, 0.001, 0.276, 0.001]


In [7]:
def num_nul_list(num,x):
    res = []
    for i in range(len(x)):
        res.append(num*x[i])
    return res
def list_sum(x,y):
    sum = []
    for i in range(len(x)):
        sum.append(x[i]+y[i])
    return sum
def cal_w(a,label,x):
    res = []
    for i in range(len(x[0])):
        res.append(0.0)
    for i in range(len(a)):
        res=list_sum(res, num_nul_list(a[i]*label[i],x[i]))
    return res
#print (a,label,x)
w = cal_w(a,label,x)
print (w)

[0.2795, 0.6930000000000001]


In [8]:
def cal_b(w,x,label):
    sum = 0.0
    for i in range(len(x)):
        sum = sum +(label[i]-sums(mul(w,x[i])))
    # 简单的取平均
    return sum/len(x)
b = cal_b(w,x,label)

In [9]:
test_x,test_label = load_x_and_label(test_data)
#print(test_x,test_label)
def f(w,b,test_x):
    predict=[]
    for i in range(len(test_x)):
        s = sums(mul(w,test_x[i]))+b
        if(s>0):
            predict.append(1.0)
        else:
            predict.append(-1.0)
    return predict

In [10]:
predict = f(w,b,test_x)
print(predict,test_label)

[-1.0, 1.0] [-1.0, 1.0]
